In [4]:
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, HumanMessage
from pydantic import BaseModel, Field
from typing import List, Optional, Union

# Define the state using Pydantic
class AgentState(BaseModel):
    input: Optional[str] = None
    messages: List[Union[HumanMessage, AIMessage]] = Field(default_factory=list)
    step: int = 0
    validation_status: Optional[str] = None
    intermediate_result: Optional[str] = None
    output: Optional[str] = None

# Simple pass-through nodes
def supervisor_node(state: AgentState):
    return state

def router_node(state: AgentState):
    return state

# Processing nodes
def llm_node(state: AgentState):
    query = state.input or ""
    response = f"[LLM Response to] {query}"
    return {
        "intermediate_result": response,
        "messages": state.messages + [AIMessage(content=response)]
    }

def rag_node(state: AgentState):
    query = state.input or ""
    response = f"RAG Retrieved Data for: {query}"
    return {
        "intermediate_result": response,
        "messages": state.messages + [AIMessage(content=response)]
    }

def web_crawler_node(state: AgentState):
    query = state.input or ""
    response = f"Web Crawler fetched data for: {query}"
    return {
        "intermediate_result": response,
        "messages": state.messages + [AIMessage(content=response)]
    }

def validation_node(state: AgentState):
    content = state.intermediate_result or ""
    if "fail" in content.lower():
        return {"validation_status": "fail"}
    return {"validation_status": "pass"}

def final_output_node(state: AgentState):
    return {"output": f"Final Validated Answer: {state.intermediate_result}"}

# Build the workflow graph
graph = StateGraph(AgentState)

# Add all nodes
graph.add_node("supervisor_node", supervisor_node)
graph.add_node("router_node", router_node)
graph.add_node("llm_node", llm_node)
graph.add_node("rag_node", rag_node)
graph.add_node("web_crawler_node", web_crawler_node)
graph.add_node("validation_node", validation_node)
graph.add_node("final_output_node", final_output_node)

# Set entry point
graph.set_entry_point("supervisor_node")

# First edge (supervisor to router)
graph.add_edge("supervisor_node", "router_node")

# Conditional routing from router
graph.add_conditional_edges(
    "router_node",
    lambda state: (
        "web_crawler_node" if ("latest" in (state.input or "").lower() or "current" in (state.input or "").lower())
        else "rag_node" if ("report" in (state.input or "").lower() or "internal" in (state.input or "").lower())
        else "llm_node"
    ),
    {
        "web_crawler_node": "web_crawler_node",
        "rag_node": "rag_node",
        "llm_node": "llm_node"
    }
)

# Processing paths
graph.add_edge("llm_node", "validation_node")
graph.add_edge("rag_node", "validation_node")
graph.add_edge("web_crawler_node", "validation_node")

# Validation conditional edges
graph.add_conditional_edges(
    "validation_node",
    lambda state: state.validation_status,
    {
        "pass": "final_output_node",
        "fail": "supervisor_node"  # Loop back on failure
    }
)

# Set finish point
graph.set_finish_point("final_output_node")

# Compile the workflow
workflow = graph.compile()

# Test the workflow
test_cases = [
    "Tell me the latest news",  # -> web crawler
    "Show me the internal report",  # -> RAG
    "Explain quantum computing",  # -> LLM
]

for user_query in test_cases:
    print(f"\nProcessing query: '{user_query}'")
    initial_state = AgentState(
        input=user_query,
        messages=[HumanMessage(content=user_query)]
    )
    result = workflow.invoke(initial_state)
    print("Result:", result["output"])


Processing query: 'Tell me the latest news'
Result: Final Validated Answer: Web Crawler fetched data for: Tell me the latest news

Processing query: 'Show me the internal report'
Result: Final Validated Answer: RAG Retrieved Data for: Show me the internal report

Processing query: 'Explain quantum computing'
Result: Final Validated Answer: [LLM Response to] Explain quantum computing
